In [1]:
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import geopandas as gpd
from shapely.geometry import box
from rasterio.crs import CRS
from rasterio.warp import transform_bounds
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rioxarray
import xarray
from rioxarray.merge import merge_arrays
import os
from osgeo import gdal

In [7]:
bucket_name = 'opera-pst-rs-pop1'
local_path = "C:/Users/Matthew Bonnema/Documents/dswx_data/wtr/"
local_reproj = "C:/Users/Matthew Bonnema/Documents/dswx_data/wtr_reproj/"
local_dest = "C:/Users/Matthew Bonnema/Documents/dswx_data/mosaic_final/"

In [3]:
with open('dswx_manifest.txt') as f:
    lines = f.readlines()

print(lines[0])

s3://opera-pst-rs-pop1/products/OPERA_L3_DSWx_HLS_T01GDM_20220101T214749Z_20221125T202742Z_S2B_30_v0.0/



In [11]:
cont = 'na' # af,as,au,na,sa,eu
lvl = '01'

In [12]:
basin_path  = f'./shapefiles/basins/hybas_{cont}/hybas_{cont}_lev{lvl}_v1c.shp'
basin_gdf = gpd.read_file(basin_path)
mrg_gdf = gpd.read_file('./shapefiles/world_mgrs/mgrs_region.shp') 


In [13]:
target_basin =  basin_gdf
grid_codes = gpd.sjoin(mrg_gdf,target_basin)
tile_list = []
for row in grid_codes.iterrows():
    tile_list.append(row[1].iloc[0]+row[1].iloc[1])
len(tile_list)

2632

In [14]:
keys = []
for line in lines:
    prefix = line.split(bucket_name+'/')[-1][:-2]
    filename = prefix.split('/')[-1]+'_B01_WTR.tiff'#'_B06_WTR-2.tiff'#'_B01_WTR.tiff'
    tile = filename.split('_')[4][1:]
    if tile in tile_list:
        keys.append(filename)
len(keys)

6808

In [15]:
i = 0
reporj_list = os.listdir(local_reproj)
for key in keys:
    try:
        input_raster = f'{local_path}{key}'
        dest_path = f'{local_reproj}{key}'
        if key in reporj_list:
            continue
        xds = rioxarray.open_rasterio(input_raster,cache=False)
        reproj = xds.rio.reproject("EPSG:4326")
        reproj.rio.to_raster(dest_path, tiled=True, windowed=True,compress='deflate')
        xds = None
        reproj = None
    except Exception as e:
        print(f'failed to reproject: {key}, error: {e}')
    if i%100 == 0:
        print(f'Finshed reprojecting {i} tiles')
    i = i+1

Finshed reprojecting 0 tiles
Finshed reprojecting 100 tiles
Finshed reprojecting 200 tiles
Finshed reprojecting 300 tiles
Finshed reprojecting 400 tiles
Finshed reprojecting 500 tiles
Finshed reprojecting 600 tiles
Finshed reprojecting 700 tiles
Finshed reprojecting 800 tiles
Finshed reprojecting 900 tiles
Finshed reprojecting 1000 tiles
Finshed reprojecting 1100 tiles
Finshed reprojecting 1200 tiles
Finshed reprojecting 1300 tiles
Finshed reprojecting 1400 tiles
Finshed reprojecting 1500 tiles
Finshed reprojecting 1600 tiles
Finshed reprojecting 1700 tiles
Finshed reprojecting 1800 tiles
Finshed reprojecting 1900 tiles
Finshed reprojecting 2000 tiles
Finshed reprojecting 2100 tiles
Finshed reprojecting 2200 tiles
Finshed reprojecting 2300 tiles
Finshed reprojecting 2400 tiles
Finshed reprojecting 2500 tiles
Finshed reprojecting 2600 tiles
Finshed reprojecting 2700 tiles
Finshed reprojecting 2800 tiles
Finshed reprojecting 2900 tiles
Finshed reprojecting 3000 tiles
Finshed reprojecting